## HITI off-target analysis

In [1]:
#bcl2fastq -i ./Data/Intensities/BaseCalls/ -R ./ -o ./demultiplexed/ --use-bases-mask Y75,I8,I8,Y75 --sample-sheet ./20220427_SampleSheet_HKVHYAFX3.csv -r 8 -p 24 -w 8 --no-lane-splitting --minimum-trimmed-read-length 10 --mask-short-adapter-reads 10 --ignore-missing-bcl --ignore-missing-filter --ignore-missing-position
#bcl2fastq -i ./Data/Intensities/BaseCalls/ -R ./ -o ./demultiplexed_noIDX/ --use-bases-mask Y75,n8,n8,Y75 --sample-sheet ./20220427_SampleSheet_HKVHYAFX3_noIDX.csv -r 8 -p 24 -w 8 --no-lane-splitting --minimum-trimmed-read-length 10 --mask-short-adapter-reads 10 --ignore-missing-bcl --ignore-missing-filter --ignore-missing-position

In [2]:
import os
import warnings
from subprocess import call
import subprocess
import glob
import pandas as pd
import tempfile
import ntpath
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
#Specify input directories
in_path = "'/media/Dropbox/MNM projects/KAW BRAVE2glia project/Experiments/Sequencing/220426 NextSeq Arc Glia AAV/MNM220426/220426_NB502004_0185_AHKVHYAFX3/demultiplexed/HITI_Arc/'"
out_path = "'/media/Dropbox/MNM projects/KAW BRAVE2glia project/Data analysis/220427_HITI_Arc_offTarget'"
base_path = '/tmp/indata'
export_path = '/tmp/outdata'

In [4]:
#the symlinks are generated to ensure path without spaces which bbmap cannot handle
!mkdir $out_path
!rm $base_path
!ln -s $in_path $base_path
!rm $export_path
!ln -s $out_path $export_path
base_path = base_path +'/'
export_path = export_path + '/'

mkdir: cannot create directory ‘/media/Dropbox/MNM projects/KAW BRAVE2glia project/Data analysis/220427_HITI_Arc_offTarget’: File exists


### Trim Barcode

In [5]:
def trimBC(sample_id) :
    print(sample_id)
    #sample_id = trimSamples_list[10]
    inPathR2 = base_path + ntpath.basename(sample_id).replace('_R1_001.fastq.gz','_R2_001.fastq.gz')
    outPathR1 = export_path + ntpath.basename(sample_id).replace('_R1_001.fastq.gz','_{name}_R1.fastq.gz')
    outPathR2 = export_path + ntpath.basename(sample_id).replace('_R1_001.fastq.gz','_{name}_R2.fastq.gz')
    
    trimLog = export_path + ntpath.basename(sample_id).replace('_R1_001.fastq.gz','_HITI-Log.tsv')
    param = "--cores=0 -g SP1_3p=ggcggcatggacgagctgtacaagccgaac -g SP1_5p=ttgctcaccatggtggcgcg --report=minimal"
    call_sequence = "cutadapt "+param+" -o "+outPathR2+" -p " + outPathR1 + " "+ inPathR2 + " " + sample_id +" > " + trimLog
    !$call_sequence
    print(call_sequence)
    #call_sequence   

In [6]:
samples_list = sorted(glob.glob(base_path+'*R1*.fastq.gz'))
samples_list= [item for item in samples_list if 'offTarget' in item]
samples_list

['/tmp/indata/HITI_offTarget_3-5p_18_S23_R1_001.fastq.gz',
 '/tmp/indata/HITI_offTarget_3-5p_18_S510_S24_R1_001.fastq.gz',
 '/tmp/indata/HITI_offTarget_pre_18_S21_R1_001.fastq.gz',
 '/tmp/indata/HITI_offTarget_pre_18_S510_S22_R1_001.fastq.gz']

In [7]:
[trimBC(sample_id) for sample_id in samples_list]

/tmp/indata/HITI_offTarget_3-5p_18_S23_R1_001.fastq.gz
Done           00:00:00           436 reads @ 434.9 µs/read;   0.14 M reads/minute
cutadapt --cores=0 -g SP1_3p=ggcggcatggacgagctgtacaagccgaac -g SP1_5p=ttgctcaccatggtggcgcg --report=minimal -o /tmp/outdata/HITI_offTarget_3-5p_18_S23_{name}_R2.fastq.gz -p /tmp/outdata/HITI_offTarget_3-5p_18_S23_{name}_R1.fastq.gz /tmp/indata/HITI_offTarget_3-5p_18_S23_R2_001.fastq.gz /tmp/indata/HITI_offTarget_3-5p_18_S23_R1_001.fastq.gz > /tmp/outdata/HITI_offTarget_3-5p_18_S23_HITI-Log.tsv
/tmp/indata/HITI_offTarget_3-5p_18_S510_S24_R1_001.fastq.gz
Done           00:00:13     3,771,658 reads @   3.5 µs/read;  17.14 M reads/minute
cutadapt --cores=0 -g SP1_3p=ggcggcatggacgagctgtacaagccgaac -g SP1_5p=ttgctcaccatggtggcgcg --report=minimal -o /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_{name}_R2.fastq.gz -p /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_{name}_R1.fastq.gz /tmp/indata/HITI_offTarget_3-5p_18_S510_S24_R2_001.fastq.gz /tmp/indata/HITI_off

[None, None, None, None]

In [8]:
samples_list = sorted(glob.glob(export_path+'*R1.fastq.gz'))
samples_list = [item for item in samples_list if 'Trimmed' not in item]
samples_list = [item for item in samples_list if 'offTarget' in item]
samples_list

['/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_5p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_3-5p_18_S23_unknown_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_3p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_5p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S21_SP1_3p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S21_SP1_5p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S21_unknown_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_SP1_3p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_SP1_5p_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_R1.fastq.gz']

In [9]:
def trimShortBB(sample_id) :
    print(sample_id)
    #sample_id = trimSamples_list[10]
    inPathR2 = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_R2.fastq.gz')
    outPathR1 = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_Trimmed_R1.fastq.gz')
    outPathR2 = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_Trimmed_R2.fastq.gz')
    
    param = "ktrim=r k=12 mink=11 hdist=2 rcomp=f"
    call_sequence = "bbduk.sh "+param+"  ref="+export_path+"adapters.fasta in1=" + sample_id + " in2=" + inPathR2 + " out1="+ outPathR1 + " out2="+ outPathR2
    !$call_sequence
    print(call_sequence)
    #call_sequence   

In [10]:
[trimShortBB(sample_id) for sample_id in samples_list]

/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R1.fastq.gz
java -ea -Xmx89600m -Xms89600m -cp /usr/local/bin/bbmap/current/ jgi.BBDuk ktrim=r k=12 mink=11 hdist=2 rcomp=f ref=/tmp/outdata/adapters.fasta in1=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R1.fastq.gz in2=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R2.fastq.gz out1=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_Trimmed_R1.fastq.gz out2=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_Trimmed_R2.fastq.gz
Executing jgi.BBDuk [ktrim=r, k=12, mink=11, hdist=2, rcomp=f, ref=/tmp/outdata/adapters.fasta, in1=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R1.fastq.gz, in2=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R2.fastq.gz, out1=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_Trimmed_R1.fastq.gz, out2=/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_Trimmed_R2.fastq.gz]
Version 38.93

maskMiddle was disabled because useShortKmers=true
0.036 seconds.
Initial:
Memory: max=93952m, total=93952m, free=93868m, used=84m

Added 4153

[None, None, None, None, None, None, None, None, None, None, None, None]

In [11]:
samples_list = sorted(glob.glob(export_path+'*R1.fastq.gz'))
for sample_file in samples_list :
    print(sample_file)
    result = subprocess.run(['pigz -dc '+ sample_file+ ' | wc -l'], stdout=subprocess.PIPE, shell=True)
    sampleReads = int(int(result.stdout)/4)
    if sampleReads < 300 :
        os.remove(sample_file)
    print(sampleReads)

/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_R1.fastq.gz
101
/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_3p_Trimmed_R1.fastq.gz
0
/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_5p_R1.fastq.gz
85
/tmp/outdata/HITI_offTarget_3-5p_18_S23_SP1_5p_Trimmed_R1.fastq.gz
0
/tmp/outdata/HITI_offTarget_3-5p_18_S23_unknown_R1.fastq.gz
250
/tmp/outdata/HITI_offTarget_3-5p_18_S23_unknown_Trimmed_R1.fastq.gz
0
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_3p_R1.fastq.gz
1075225
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_3p_Trimmed_R1.fastq.gz
200
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_5p_R1.fastq.gz
782398
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_5p_Trimmed_R1.fastq.gz
30
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_R1.fastq.gz
1914035
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz
581
/tmp/outdata/HITI_offTarget_pre_18_S21_SP1_3p_R1.fastq.gz
0
/tmp/outdata/HITI_offTarget_pre_18_S21_SP1_3p_Trimmed_R1.fastq.gz
0
/tmp/outdata/HITI_offTarget_pre

In [12]:
qc_list = sorted(glob.glob(export_path+'*HITI-Log.tsv'))
allSampleTable = pd.DataFrame()

for qc_file in qc_list :
    df=pd.read_csv(qc_file, sep='\t') # , header=0, index_col=0,skiprows=None
    df = df.set_index(pd.Series(ntpath.basename(qc_file).replace('_trimLog.tsv','')))
    allSampleTable = pd.concat([allSampleTable, df], axis=0)

allSampleTable

,status,in_reads,in_bp,too_short,too_long,too_many_n,out_reads,w/adapters,qualtrim_bp,out_bp,w/adapters2,qualtrim2_bp,out2_bp
HITI_offTarget_3-5p_18_S23_HITI-Log.tsv,OK,436,65400,0,0,0,436,186,0,25985,0,0,32700
HITI_offTarget_3-5p_18_S510_S24_HITI-Log.tsv,OK,3771658,565748700,0,0,0,3771658,1857623,0,216504223,0,0,282874350
HITI_offTarget_pre_18_S21_HITI-Log.tsv,OK,74,11100,0,0,0,74,0,0,5550,0,0,5550
HITI_offTarget_pre_18_S510_S22_HITI-Log.tsv,OK,3374698,506204700,0,0,0,3374698,3,0,253102238,0,0,253102350


In [13]:
def trimShort(sample_id) :
    print(sample_id)
    #sample_id = trimSamples_list[10]
    inPathR2 = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_R2.fastq.gz')
    outPathR1 = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_Trimmed_R1.fastq.gz')
    outPathR2 = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_Trimmed_R2.fastq.gz')
    
    trimLog = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_Trim-Log.tsv')
    param = "--cores=0 -a Trim=aggctaagaactcctccgcgccacc -a Trim2=atggtgagcaagggcgaggaggataacatgg -a Trim3=TAGATCGGAAGAGCACACGTCTGAACTCCAG --report=minimal -n 5 -m=30"
    call_sequence = "cutadapt "+param+" -o "+outPathR1+" -p " + outPathR2 + " "+ sample_id + " " + inPathR2 +" > " + trimLog
    !$call_sequence
    print(call_sequence)
    #call_sequence   

In [15]:
samples_list = sorted(glob.glob(export_path+'*Trimmed_R1.fastq.gz'))
samples_list

['/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_R1.fastq.gz']

In [16]:
[trimShort(sample_id) for sample_id in samples_list]

/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz
Done           00:00:00           581 reads @ 304.5 µs/read;   0.20 M reads/minute
cutadapt --cores=0 -a Trim=aggctaagaactcctccgcgccacc -a Trim2=atggtgagcaagggcgaggaggataacatgg -a Trim3=TAGATCGGAAGAGCACACGTCTGAACTCCAG --report=minimal -n 5 -m=30 -o /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_Trimmed_R1.fastq.gz -p /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_Trimmed_R2.fastq.gz /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R2.fastq.gz > /tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_Trim-Log.tsv
/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_R1.fastq.gz
Done           00:00:01       106,147 reads @   9.7 µs/read;   6.16 M reads/minute
cutadapt --cores=0 -a Trim=aggctaagaactcctccgcgccacc -a Trim2=atggtgagcaagggcgaggaggataacatgg -a Trim3=TAGATCGGAAGAGCACACGTCTGAACTCCAG -

[None, None]

In [17]:
samples_list = sorted(glob.glob(export_path+'*R1.fastq.gz'))
for sample_file in samples_list :
    print(sample_file)
    result = subprocess.run(['pigz -dc '+ sample_file+ ' | wc -l'], stdout=subprocess.PIPE, shell=True)
    sampleReads = int(int(result.stdout)/4)
    if sampleReads < 300 :
        os.remove(sample_file)
    print(sampleReads)

/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_3p_R1.fastq.gz
1075225
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_SP1_5p_R1.fastq.gz
782398
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_R1.fastq.gz
1914035
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz
581
/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_Trimmed_R1.fastq.gz
1
/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_R1.fastq.gz
3374695
/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_R1.fastq.gz
106147
/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_Trimmed_R1.fastq.gz
1026


In [18]:
Trimmed_list = sorted(glob.glob(export_path+'*Trimmed_R1.fastq.gz'))
Trimmed_list

['/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_R1.fastq.gz',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_Trimmed_R1.fastq.gz']

In [19]:
def runSC(sample_id) :
    print(sample_id)
    
    outPath = export_path + ntpath.basename(sample_id).replace('_R1.fastq.gz','_starcode.tsv')
    starcode_call= "gunzip -c '"+ sample_id +"' | /media/data/TomasB/starcode/starcode -t 48 -o '"+outPath + "'"
    !$starcode_call 

In [20]:
[runSC(sample_id) for sample_id in Trimmed_list]

/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_R1.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering
/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_R1.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 2
progress: 100.00%
message passing clustering
/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_Trimmed_R1.fastq.gz
running starcode with 48 threads
reading input files
FASTQ format detected
sorting
setting dist to 3
progress: 100.00%
message passing clustering


[None, None, None]

In [21]:
Trimmed_SC_list = sorted(glob.glob(export_path+'*_starcode.tsv'))
Trimmed_SC_list

['/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_starcode.tsv',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_Trimmed_starcode.tsv',
 '/tmp/outdata/HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_starcode.tsv']

In [22]:
starcodeTable = pd.DataFrame()
sample_id = '/tmp/outdata/HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed_starcode.tsv'
df = pd.read_csv(sample_id, sep='\t', header=None, index_col=0, dtype={0: str, 1: 'Int64'})
df

,1
0,
GTGATAGAGGATGCCA,83
CTCATAATGAG,47
GTACAAGCCGA,29
GTACAAGCCG,21
TGATAGAGGATGCCA,18
...,...
TGACTACGCAG,1
TGATCATAGA,1
TGGCTTCCCC,1


In [23]:
starcodeTable = pd.DataFrame()
for sample_id in Trimmed_SC_list :
    df = pd.read_csv(sample_id, sep='\t', header=None, index_col=0, dtype={0: str, 1: 'Int64'})
    df = df.rename(columns={1: ntpath.basename(sample_id).replace('_starcode.tsv','')})
    starcodeTable = starcodeTable.merge(df,left_index=True, right_index=True, how='outer')

In [ ]:
starcodeTable

,HITI_offTarget_3-5p_18_S510_S24_unknown_Trimmed,HITI_offTarget_pre_18_S510_S22_unknown_Trimmed_Trimmed,HITI_offTarget_pre_18_S510_S22_unknown_Trimmed
0,,,
AAAAAAAAGACT,<NA>,<NA>,1
AAAAAAACATACT,<NA>,<NA>,1
AAAAAACACCCTAC,<NA>,<NA>,1
AAAAAACCCTGT,<NA>,<NA>,1
AAAAAAGAGAC,<NA>,<NA>,1
...,...,...,...
TTTTTGTATA,<NA>,<NA>,1
TTTTTGTCCC,<NA>,<NA>,1
TTTTTTAGGCACA,<NA>,<NA>,1
